In [1]:
import pandas as pd
import os
from glob import glob
import re

In [2]:
path = r"C:\Users\sachi\Documents\Researchcode\master_combined_risk"
csv_files = glob(os.path.join(path, "*.csv"))
combined_dfs = []
for file in csv_files:
    filename = os.path.basename(file).replace(".csv", "")
    lag_match = re.search(r'_(\d+)_', filename)
    Type = filename.strip().split('_')[-1]
    lag = int(lag_match.group(1)) if lag_match else None    
    df = pd.read_csv(file)
    df['lag'] = lag
    df['Type'] = Type
    combined_dfs.append(df)
df = pd.concat(combined_dfs, ignore_index=True)
df.rename(columns = {'Unnamed: 0':'Ticker'}, inplace = True)
def clean_df(data):
    data = data[['Ticker','Model','lag', 'Accuracy_x', 'Confusion Matrix_x','Return [%]_x','Buy & Hold Return [%]_x','Return [%]_y', 'Type']]
    data.reset_index(inplace = True,drop = True)
    data.loc[12,'Accuracy_x'] = 1
    data.loc[25,'Accuracy_x'] = 1
    data.loc[38,'Accuracy_x'] = 1
    data.loc[12,'Confusion Matrix_x'] = 1
    data.loc[25,'Confusion Matrix_x'] = 1
    data.loc[38,'Confusion Matrix_x'] = 1


    def get_metrics(data):
        def extract_matrix_values(matrix_str):
            matrix_str = str(matrix_str)
            numbers = re.findall(r'\d+', matrix_str)
            if len(numbers) < 4:
                return pd.Series({'TN': 1, 'FP': 1, 'FN': 1, 'TP': 1})
            
            numbers = [int(x) for x in numbers]
            
            return pd.Series({
                'TN': numbers[0],
                'FP': numbers[1],
                'FN': numbers[2],
                'TP': numbers[3]
            })
        new_columns = data['Confusion Matrix_x'].apply(extract_matrix_values)
        data[['TN', 'FP', 'FN', 'TP']] = new_columns
        
        def calculate_metrics(row):
            tp = row.get('TP', 0)
            fp = row.get('FP', 0)
            fn = row.get('FN', 0)
            tn = row.get('TN', 0)

            precision = tp / (tp + fp) if (tp + fp) != 0 else 0
            tpr = tp / (tp + fn) if (tp + fn) != 0 else 0  # True Positive Rate (Recall)
            fpr = fp / (fp + tn) if (fp + tn) != 0 else 0  # False Positive Rate

            return pd.Series({
                'Precision': precision,
                'TPR': tpr,         
                'FPR': fpr
            })

        
        metrics = data.apply(calculate_metrics, axis=1)
        data = pd.concat([data, metrics], axis=1)
        data.drop(columns = ['Confusion Matrix_x'],inplace = True)
        return data

    data = get_metrics(data)
    return data


In [3]:
data = clean_df(df)

C:\Users\sachi\AppData\Local\Temp\ipykernel_23568\3237335672.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[['TN', 'FP', 'FN', 'TP']] = new_columns
C:\Users\sachi\AppData\Local\Temp\ipykernel_23568\3237335672.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[['TN', 'FP', 'FN', 'TP']] = new_columns
C:\Users\sachi\AppData\Local\Temp\ipykernel_23568\3237335672.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [4]:
data.shape

(5070, 15)

In [5]:
data = data[data['Ticker'] == 'META']

In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [7]:
data.columns

Index(['Ticker', 'Model', 'lag', 'Accuracy_x', 'Return [%]_x',
       'Buy & Hold Return [%]_x', 'Return [%]_y', 'Type', 'TN', 'FP', 'FN',
       'TP', 'Precision', 'TPR', 'FPR'],
      dtype='object')

In [8]:
data.shape

(507, 15)

In [9]:
data

,Ticker,Model,lag,Accuracy_x,Return [%]_x,Buy & Hold Return [%]_x,Return [%]_y,Type,TN,FP,FN,TP,Precision,TPR,FPR
52,META,Logistic Regression,0,0.824306,299.688779,225.732375,291.285394,grid,182,72,61,442,0.859922,0.878728,0.283465
53,META,Random Forest,0,0.838838,215.088994,225.732375,214.610687,grid,202,52,70,433,0.892784,0.860835,0.204724
54,META,XGBoost,0,0.844122,233.636010,225.732375,242.572696,grid,197,57,61,442,0.885772,0.878728,0.224409
55,META,SVM,0,0.494055,36.151189,225.732375,32.421416,grid,229,25,358,145,0.852941,0.288270,0.098425
56,META,Naive Bayes,0,0.820343,114.713403,225.732375,110.458598,grid,211,43,93,410,0.905077,0.815109,0.169291
57,META,Gradient Boosting,0,0.833554,198.835953,225.732375,184.843340,grid,210,44,82,421,0.905376,0.836978,0.173228
58,META,K-Nearest Neighbors,0,0.789960,115.260297,225.732375,109.495419,grid,199,55,104,399,0.878855,0.793241,0.216535
59,META,Decision Tree,0,0.820343,194.522787,225.732375,192.559066,grid,216,38,98,405,0.914221,0.805169,0.149606
60,META,AdaBoost,0,0.857332,244.690305,225.732375,223.963902,grid,206,48,60,443,0.902240,0.880716,0.188976
61,META,SGD Classifier,0,0.824306,128.296833,225.732375,128.055954,grid,234,20,113,390,0.951220,0.775348,0.078740
